In [129]:
import requests
import json
import keys
from alpha_vantage.foreignexchange import ForeignExchange
import numpy as np

In [80]:
def get_gold_price(api_key):
    fx = ForeignExchange(key=api_key)
    result, _ = fx.get_currency_exchange_rate(from_currency='XAU', to_currency='USD')
    return result['9. Ask Price']

In [81]:
api_key = keys.ALPHA_VANTAGE
result = get_gold_price(api_key)
print(json.dumps(result, indent=4))

"1940.61000000"


In [189]:
#Add a file keys.py in the folder, and input your own API KEYS

class WiseClient:
    def __init__(self, access_token):
        self.access_token = access_token
        self.base_url = "https://api.sandbox.transferwise.tech/"
        self.headers = {
            'Authorization': f'Bearer {self.access_token}',
            'Content-Type': 'application/json'
        }
        self.gold = float(get_gold_price(api_key))
    def generate_uuid(self):
        url = "https://www.uuidgenerator.net/api/guid"

        payload={}
        headers = {}

        response = requests.request("GET", url, headers=headers, data=payload)

        return response.text

        
    def get_balance(self, profile_id):
        url = self.base_url + f'v1/borderless-accounts?profileId={profile_id}'
        response = requests.get(url, headers=self.headers)
        balance_data = response.json()
        return balance_data  # This should return your account balances in different currencies

    def get_rates(self,source,target):
        url = self.base_url + f'/v1/rates?source={source}&target={target}'
        response = requests.get(url,headers=self.headers)
        return response.json()
    
    def get_true_change_target(self,source,target,targetAmount):
        url = self.base_url + f'/v1/quotes?source={source}&target={target}&rateType=FIXED&targetAmount={targetAmount}'
        response = requests.get(url,headers=self.headers)
        return response.json()
    
    def get_true_change_source(self,source,target,sourceAmount):
        url = self.base_url + f'/v1/quotes?source={source}&target={target}&rateType=FIXED&sourceAmount={sourceAmount}'
        response = requests.get(url,headers=self.headers)
        return response.json()

    def create_quote(self, profile_id, source_currency, target_currency, amount):
        url = self.base_url + 'v1/quotes'
        quote_payload = {
            'profile': profile_id,
            'source': source_currency,
            'target': target_currency,
            'sourceAmount': amount,
            'rateType': 'FIXED',
            'type': 'BALANCE_CONVERSION'
        }
        response = requests.post(url, headers=self.headers, data=json.dumps(quote_payload))
        quote_data = response.json()
        return quote_data  # This should return the details of the quote including its id

    def check_id(self,account_id):
        url = self.base_url+ '/v1/accounts/' + account_id
        response = requests.get(url, headers=self.headers)
        return response.json()

    def initiate_transfer(self, target_account_id, quote_id, reference):
        url = self.base_url + 'v1/transfers'
        transfer_payload = {
            'targetAccount': target_account_id,
            'quote': quote_id,
            'customerTransactionId': self.generate_uuid(),
            'details': {
                'reference': reference,
                'transferPurpose': 'verification.transfers.purpose.pay.bills'
            }
        }
        response = requests.post(url, headers=self.headers, data=json.dumps(transfer_payload))
        transfer_data = response.json()
        return transfer_data  
    
    def make_payment(self, profile_id, transfer_id):
        url = self.base_url + f'v3/profiles/{profile_id}/transfers/{transfer_id}/payments'
        payment_payload = {
            'type': 'BALANCE',
        }
        response = requests.post(url, headers=self.headers, data=json.dumps(payment_payload))
        payment_data = response.json()
        return payment_data
    
    def TAL_weights(self,source):
        weights = np.array([
            self.get_rates('CHF', source)[0]['rate'] * 100,
            self.get_rates('EUR', source)[0]['rate']  * 250,
            self.get_rates('GBP', source)[0]['rate']  * 50,
            self.get_rates('JPY', source)[0]['rate']  * 18000,
            self.get_rates('CNY', source)[0]['rate']  * 1600,
            self.get_rates('SGD', source)[0]['rate']  * 80,
            self.get_rates('USD',source)[0]['rate']  * self.gold*0.2
        ])

        response = json.loads(self.TAL_rate(source))
        
        weights /= response['total_amount']

        return weights
    
    def TAL_rate(self,source):
        chf = self.get_true_change_target(source,'CHF',100)
        chf_amount = chf['sourceAmount']
        chf_fee = chf['fee']
        eur = self.get_true_change_target(source,'EUR',250)
        eur_amount = eur['sourceAmount']
        eur_fee = eur['fee']
        gbp = self.get_true_change_target(source,'GBP',50)
        gbp_amount = gbp['sourceAmount']
        gbp_fee = gbp['fee']
        jpy = self.get_true_change_target(source,'JPY',18000)
        jpy_amount = jpy['sourceAmount']
        jpy_fee = jpy['fee']
        cny = self.get_true_change_target(source,'CNY',1600)
        cny_amount = cny['sourceAmount']
        cny_fee = cny['fee']
        sgd = self.get_true_change_target(source,'SGD',80)
        sgd_amount = sgd['sourceAmount']
        sgd_fee = sgd['fee']
        gold = self.gold*0.2
        usdgold = self.get_true_change_target(source,'USD',gold)
        usdgold_amount = usdgold['sourceAmount']
        usdgold_fee = usdgold['fee']

        result_dict = {
            'total_amount': chf_amount + eur_amount + gbp_amount + jpy_amount + cny_amount + sgd_amount + usdgold_amount,
            'total_fee': chf_fee + eur_fee + gbp_fee + jpy_fee + cny_fee + sgd_fee + usdgold_fee,
            'chf': {'amount': chf_amount, 'fee': chf_fee},
            'eur': {'amount': eur_amount, 'fee': eur_fee},
            'gbp': {'amount': gbp_amount, 'fee': gbp_fee},
            'jpy': {'amount': jpy_amount, 'fee': jpy_fee},
            'cny': {'amount': cny_amount, 'fee': cny_fee},
            'sgd': {'amount': sgd_amount, 'fee': sgd_fee},
            'usdgold': {'amount': usdgold_amount, 'fee': usdgold_fee}
        }
        
        # Convert the dictionary to a JSON string
        result_json = json.dumps(result_dict,indent=4)
        
        return result_json

    def True_TAL_Change(self,source,sourceAmount):
        weights = self.TAL_weights(source)

        distribution = sourceAmount*weights

        chf = self.get_true_change_source(source,'CHF',distribution[0])
        chf_amount = chf['targetAmount']
        chf_fee = chf['fee']
        eur = self.get_true_change_source(source,'EUR',distribution[1])
        eur_amount = eur['targetAmount']
        eur_fee = eur['fee']
        gbp = self.get_true_change_source(source,'GBP',distribution[2])
        gbp_amount = gbp['targetAmount']
        gbp_fee = gbp['fee']
        jpy = self.get_true_change_source(source,'JPY',distribution[3])
        jpy_amount = jpy['targetAmount']
        jpy_fee = jpy['fee']
        cny = self.get_true_change_source(source,'CNY',distribution[4])
        cny_amount = cny['targetAmount']
        cny_fee = cny['fee']
        sgd = self.get_true_change_source(source,'SGD',distribution[5])
        sgd_amount = sgd['targetAmount']
        sgd_fee = sgd['fee']
        usdgold = self.get_true_change_source(source,'USD',distribution[6])
        gold_amount = usdgold['targetAmount'] / self.gold
        gold_fee = usdgold['fee']

        result_dict = {
            'total_fee': chf_fee + eur_fee + gbp_fee + jpy_fee + cny_fee + sgd_fee + gold_fee,
            'chf': {'amount': chf_amount, 'fee': chf_fee},
            'eur': {'amount': eur_amount, 'fee': eur_fee},
            'gbp': {'amount': gbp_amount, 'fee': gbp_fee},
            'jpy': {'amount': jpy_amount, 'fee': jpy_fee},
            'cny': {'amount': cny_amount, 'fee': cny_fee},
            'sgd': {'amount': sgd_amount, 'fee': sgd_fee},
            'gold': {'amount': gold_amount, 'fee': gold_fee}
        }
        result_json = json.dumps(result_dict,indent=4)
        
        return result_json

    
wise_client = WiseClient(keys.API)

In [190]:
distribution = wise_client.True_TAL_Change('EUR',1000)
print(distribution)

{
    "total_fee": 11.39,
    "chf": {
        "amount": 86.08,
        "fee": 0.97
    },
    "eur": {
        "amount": 216.94,
        "fee": 0.63
    },
    "gbp": {
        "amount": 42.91,
        "fee": 0.71
    },
    "jpy": {
        "amount": 15230,
        "fee": 2.84
    },
    "cny": {
        "amount": 1368.5,
        "fee": 3.01
    },
    "sgd": {
        "amount": 68.27,
        "fee": 0.92
    },
    "gold": {
        "amount": 0.17274423687574744,
        "fee": 2.31
    }
}


In [181]:
weights = wise_client.TAL_weights('BRL')
weights

array([0.08806115, 0.21316821, 0.04996821, 0.09965962, 0.17213181,
       0.04623949, 0.30008921])

In [170]:
tal_rate = json.loads(wise_client.TAL_rate('BRL'))
print(tal_rate['total_amount'])

6275.700000000001


In [49]:
# https://github.com/transferwise/api-docs/blob/master/source/includes/_payouts.md

In [89]:
profile_id = keys.PROFILE_ID
balances = wise_client.get_balance(profile_id)

# Find USD and EUR balance ids
usd_balance_id = next((balance['id'] for balance in balances[0]['balances'] if balance['currency'] == 'USD'), None)
eur_balance_id = next((balance['id'] for balance in balances[0]['balances'] if balance['currency'] == 'EUR'), None)

print(balances)
print(usd_balance_id,eur_balance_id)


[{'id': 64440, 'profileId': 16841712, 'recipientId': 148941324, 'creationTime': '2023-07-11T12:27:53.899Z', 'modificationTime': '2023-07-11T12:27:53.899Z', 'active': True, 'eligible': True, 'balances': [{'id': 158818, 'balanceType': 'AVAILABLE', 'currency': 'USD', 'amount': {'value': 2999000.0, 'currency': 'USD'}, 'reservedAmount': {'value': 0.0, 'currency': 'USD'}, 'bankDetails': None}, {'id': 158819, 'balanceType': 'AVAILABLE', 'currency': 'AUD', 'amount': {'value': 4001496.24, 'currency': 'AUD'}, 'reservedAmount': {'value': 0.0, 'currency': 'AUD'}, 'bankDetails': None}, {'id': 158817, 'balanceType': 'AVAILABLE', 'currency': 'EUR', 'amount': {'value': 2000000.0, 'currency': 'EUR'}, 'reservedAmount': {'value': 0.0, 'currency': 'EUR'}, 'bankDetails': None}, {'id': 158816, 'balanceType': 'AVAILABLE', 'currency': 'GBP', 'amount': {'value': 999984.0, 'currency': 'GBP'}, 'reservedAmount': {'value': 0.0, 'currency': 'GBP'}, 'bankDetails': None}]}]
158818 158817


In [150]:
rates = wise_client.get_rates('USD','EUR')[0]['rate']
print(rates)

0.906936


In [107]:
true_change = wise_client.get_true_change('USD','EUR',1000)

print(true_change['sourceAmount'], true_change['fee'])
print(true_change)

1113.29 10.97
{'source': 'USD', 'target': 'EUR', 'sourceAmount': 1113.29, 'targetAmount': 1000.0, 'type': 'REGULAR', 'rate': 0.907175, 'createdTime': '2023-07-12T09:23:53.971Z', 'rateType': 'FIXED', 'deliveryEstimate': '2023-07-13T09:31:00Z', 'fee': 10.97, 'feeDetails': {'transferwise': 5.63, 'payIn': 5.34, 'discount': 0, 'priceSetId': 129, 'partner': 0.0}, 'allowedProfileTypes': ['PERSONAL', 'BUSINESS'], 'guaranteedTargetAmount': False, 'ofSourceAmount': False}


In [116]:
true_change = wise_client.get_true_change_source('USD','EUR',1000)
true_change

{'source': 'USD',
 'target': 'EUR',
 'sourceAmount': 1000.0,
 'targetAmount': 897.57,
 'type': 'REGULAR',
 'rate': 0.907076,
 'createdTime': '2023-07-12T09:37:52.351Z',
 'rateType': 'FIXED',
 'deliveryEstimate': '2023-07-13T09:31:00Z',
 'fee': 10.48,
 'feeDetails': {'transferwise': 5.14,
  'payIn': 5.34,
  'discount': 0,
  'priceSetId': 129,
  'partner': 0.0},
 'allowedProfileTypes': ['PERSONAL', 'BUSINESS'],
 'guaranteedTargetAmount': False,
 'ofSourceAmount': True}

In [104]:
# Create a Wise client
wise_client = WiseClient(keys.API)

# Create a quote
profile_id = keys.PROFILE_ID # replace this with your profile id
quote_data = wise_client.create_quote(profile_id, 'USD', 'EUR', 100)

balances = wise_client.get_balance(profile_id)

# Find USD and EUR balance ids
usd_balance_id = next((balance['id'] for balance in balances[0]['balances'] if balance['currency'] == 'USD'), None)
eur_balance_id = next((balance['id'] for balance in balances[0]['balances'] if balance['currency'] == 'EUR'), None)

# Initiate a transfer
target_account_id = keys.RECIPIENT_ID
reference = 'reference'  # replace this with your reference text
transfer_data = wise_client.initiate_transfer(target_account_id, quote_data['id'], reference)

payment_data = wise_client.make_payment(profile_id, transfer_data['id'])

print(payment_data)

AttributeError: module 'keys' has no attribute 'RECIPIENT_ID'